<a href="https://colab.research.google.com/github/VarshaCU/MentalHealth-Chatbot/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/VarshaCU/MentalHealth-Chatbot.git
%cd /content/MentalHealth-Chatbot


Cloning into 'MentalHealth-Chatbot'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
/content/MentalHealth-Chatbot


In [14]:
import numpy as np
import random
import json
import nltk
nltk.download('punkt_tab')
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
    return word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    sentence_words = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1
    return bag

# Load intents data
with open('/content/MentalHealth-Chatbot/intents.json','r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

model = NeuralNet(input_size, hidden_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        outputs = model(words)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Final loss: {loss.item():.4f}')

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'Training complete. Model saved to {FILE}')

# Define a function to chat with the bot
def chat_with_bot():
    print("Start chatting with the bot (type 'quit' to exit).")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break
        else:
            # Preprocess the user's input
            input_bag = bag_of_words(tokenize(user_input), all_words)
            input_tensor = torch.tensor(input_bag, dtype=torch.float32).to(device)

            # Get the model's prediction
            output = model(input_tensor)
            predicted_class = torch.argmax(output).item()
            tag = tags[predicted_class]

            # Find an appropriate response from intents.json
            for intent in intents['intents']:
                if intent['tag'] == tag:
                    responses = intent['responses']
                    response = random.choice(responses)
                    print(f"Bot: {response}")

if __name__ == "__main__":
    chat_with_bot()


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Epoch [100/1000], Loss: 0.3342
Epoch [200/1000], Loss: 0.1410
Epoch [300/1000], Loss: 0.0061
Epoch [400/1000], Loss: 0.1822
Epoch [500/1000], Loss: 0.0020
Epoch [600/1000], Loss: 0.0999
Epoch [700/1000], Loss: 0.1918
Epoch [800/1000], Loss: 0.0010
Epoch [900/1000], Loss: 0.0005
Epoch [1000/1000], Loss: 0.0003
Final loss: 0.0003
Training complete. Model saved to data.pth
Start chatting with the bot (type 'quit' to exit).
You: what is stress
Bot: Stress is the body's natural response to a perceived threat or challenge. It can be triggered by various factors, including work, relationships, financial problems, or major life changes.
You: quit
